In [1]:
import dynet as dy
import json
from glove import Glove
import numpy as np
import csv
from collections import defaultdict
from operator import itemgetter
import random


def DefaultdictInside():
    return [defaultdict(int),0]

In [2]:
vectorDim = 50 # 50 100 200 300
ngram = 2 # 3->trigram   2->bigram  1->unigram

In [3]:
# bos # begin of sentence
# eos # end of sentence
# eol # end of line
# bol # begin of line

def read_dataset(fpath):
    print("Reading dataset...")
    poems=[]
    n_grams_dict=defaultdict(DefaultdictInside)
    with open(fpath) as json_file:
        data = json.load(json_file)
        for d in data:
            if d["id"]==100:
                break
            poem = d["poem"]
            poem = poem.replace("\n"," eol bol ")
            #poem = poem.lower() -- dataset already lowercased
            #poem = poem.replace("\n"," eol ")
            poem = poem.replace("."," ")
            poem = poem.replace(":"," ")
            poem = poem.replace("?"," ")
            poem = "bos bol "+poem+" eol eos"
            #poem = "bos "+poem+" eos"
            poem = poem.split()
            poems.append([])
            for i in range(len(poem)-ngram+1):
                poems[-1].append(poem[i:i+ngram])
                
                prev_gram = poems[-1][-1][:ngram-1]
                next_gram = poems[-1][-1][-1]    
                n_grams_dict[" ".join(prev_gram)][1]+=1
                n_grams_dict[" ".join(prev_gram)][0][next_gram]+=1
            n_grams_dict["eos"][1]+=1
            
    print("Just Finished Reading dataset...")
    return poems,n_grams_dict

poems,count_dict= read_dataset('unim_poem.json')
print("Poem Count:",len(poems))
#print(poems[0])
keys = list(count_dict.keys())
unique_gram = len(keys)
print("Unique Gram:",unique_gram)
tot=0
for k,v in count_dict.items():
    tot += v[1]
print("Total Word:",tot)

# replace(".:?"," ")
# , ; ! " | yok
# ' - 're 's n't 've 'll ellemedim

Reading dataset...
Just Finished Reading dataset...
Poem Count: 100
Unique Gram: 1361
Total Word: 5223


In [4]:
def read_glove(fpath):
    print('Loading word vectors...')
    word2vec = {}
    embeds = []
    word2idx = {}
    with open(fpath, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            word2idx[word] = len(embeds)
            vec = np.asarray(values[1:], dtype='float32')
            word2vec[word] = vec
            embeds.append(vec)
            
    mean = np.array(embeds).mean(axis=0,dtype='float32')
    word2vec["mmeann"]=mean
    embeds.append(mean)
    
    print("Finished Loading word vectors...")
    return np.array(embeds),word2idx,word2vec

embedding,w2i, w2v= read_glove('glovo/glove.6B.'+str(vectorDim)+'d.txt')
#w2v["mmeann"]=embedding.mean(axis=0) # if the word doesn't occur in vocab, it will take mean value
print(embedding.shape)
print(embedding.size)

Loading word vectors...
Finished Loading word vectors...
(400001, 50)
20000050


In [23]:
def sumvec(w2v,words):
    """
        If our model was 3,4,5gram, 
        this function would add the value of the w2vs for input vector
    """
    _temp = np.zeros(50)
    for w in words:
        try:
            _temp+=w2v[w]
        except:
            _temp+=w2v["mmeann"]
    return _temp



h = 150 # HiddenUnit
m = vectorDim

EPOCH = 10

H = _model.add_parameters((h, m))
d = _model.add_parameters(h)
U = _model.add_parameters((unique_gram, h))
b = _model.add_parameters(unique_gram)
_trainer = dy.SimpleSGDTrainer(_model)


i=0
for epoch in range(1, EPOCH + 1):
    epoch_loss = 0.0
    for p in poems:
        for gram in p:
            x=sumvec(w2v,gram[:ngram-1])
            try:
                y=keys.index(gram[-1])
            except:
                y=random.randint(0,len(keys)-1)
            dy.renew_cg()
            x = dy.inputVector(x)
            input_layer = dy.tanh(H * x + d)
            hidden_layer = U * input_layer + b
            output_layer = dy.softmax(hidden_layer)
            loss = dy.pickneglogsoftmax(output_layer, y)
            epoch_loss += loss.scalar_value()
            loss.forward()
            loss.backward()
            _trainer.update()
    print("Epoch %d. loss = %f" % (epoch, epoch_loss/len(poems)))

Epoch 1. loss = 369.670079
Epoch 2. loss = 365.700067
Epoch 3. loss = 358.593593
Epoch 4. loss = 358.499232
Epoch 5. loss = 358.331282
Epoch 6. loss = 358.258912
Epoch 7. loss = 358.244593
Epoch 8. loss = 358.243319
Epoch 9. loss = 358.242166
Epoch 10. loss = 358.237965


In [67]:
# I leave you alone with my meaningless, absurd poems

# def ganchart(possiblities):
#     possiblities=sorted(possiblities,reverse=False)
#     _sum = sum(possiblities)
#     _len = len(possiblities)
#     print("SUM:",_sum)
#     print("LEN:",_len)
   
#     ind=-1
#     rnd = random.uniform(0.0, _sum)
#     print("\tRAND:",rnd)
#     curTot = 0.0
#     for i in range(_len):
#         curTot+=possiblities[i]
#         if curTot >= rnd:
#             ind=i
#             break
#     print("\t\tINDEX:",ind)
#     return ind


close=20#int(unique_gram/100*2)
satir = 2
_tempPoem=[]
generated_poems=[]
poemCount=20

for _ in range(poemCount):
    generated_poems.append([])
    _tempPoem=["bos"]
    for i in range(satir):
        #while 1:
        while len(_tempPoem)<10:
            x=sumvec(w2v,_tempPoem[-1])
            dy.renew_cg()
            x = dy.inputVector(x)
            input_layer = dy.tanh(H * x + d)
            hidden_layer = U * input_layer + b
            output_layer = list(dy.softmax(hidden_layer).value())
            oen = list(enumerate(output_layer))
            oen=sorted(oen,key=itemgetter(1),reverse=True)
            rnd = random.randint(0,close-1)
            _tempPoem.append(keys[oen[rnd][0]])
            #print(_tempPoem)
            if _tempPoem[-1]=="eol":
                break
        generated_poems[-1]+=(_tempPoem)
        _tempPoem=["eol"]
        #print()

    
    generated_poems[-1]= " ".join(generated_poems[-1]).replace("bos ","").replace("eol ","\n").replace("eol","").replace("bol ", "")
    
    #print(generated_poems[-1],"\n##############")
    


In [68]:
def Sprob(poem,n_grams_dict,unique_word,ngram):
    # Returns the MLE of given sentence with laplace smoothing
        
    result = 1
    split_poem = poem.split()+["eol"]


    for index in range(len(split_poem)-ngram):
        prev_gram = " ".join(split_poem[index:ngram+index-1])
        next_gram = split_poem[ngram+index-1]

        result *= ( (n_grams_dict[prev_gram][0][next_gram] + 1) / (n_grams_dict[prev_gram][1] + unique_word) )

    #print("\t-> S-Probabilty of sentence :{0:.30f}".format(result))
    return result



def perplexity(poems,n_grams_dict,unique_word,ngram):
    # Returns the perplexity of the given sentence
    # second formula from assignment pdf 

    
    for p in poems:
        result=1
        print(p)
        for pl in p.split("\n"):
            result*=Sprob(pl,n_grams_dict,unique_word,ngram)
        
        result=1/result
        result = result**(1/(len(p.split(" "))+satir))

        print("\t-> Perplexity of poem :{0:.20f}\n########################".format(result))

perplexity(generated_poems,count_dict,unique_gram,ngram)

the adjusted we heaven rarely another adjusted lime adjusted 
adjusted pass moon sheet delight water baby delight we
	-> Perplexity of poem :324.35789555745333245795
########################
woman's we adjusted that coin 

sheet moonlight have 
	-> Perplexity of poem :51.49990014837545260207
########################
rarely staring seems freight rarely life woman's up silver 
lot has its shrouded moon lot at north
	-> Perplexity of poem :298.79221078498949282221
########################
came been blue woman's if rarely undone undone 

saw rarely sin resonant staring stretch frail slabs frail
	-> Perplexity of poem :298.27598553501053402215
########################
missing' 

has its life my share undone we me
	-> Perplexity of poem :93.25408392245103073037
########################
came big share law law another sheet adjusted sin 
doesn't adjusted how reck sails the love deja
	-> Perplexity of poem :283.72760059364611606725
########################
came freight water staring rarely can'